In [15]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split, Subset, TensorDataset
import torch.nn.functional as F
from torch import Tensor

from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt

import os

print(os.getcwd()) # dosya yolunu ver
%run ./Model.ipynb
%run ./Dataset.ipynb

%run ./Flow_Model.ipynb

plt.rcParams['font.size'] = 14
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']

/content


In [16]:
# Hyper-Parameters & Settings
batch_size = 500

In [17]:
# Dataset
train_dataset = CIFAR10Dataset(mode='train')
val_dataset = CIFAR10Dataset(mode='val')
test_dataset = CIFAR10Dataset(mode='test')

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

In [18]:
# UAE modelini çalıştır
os.makedirs("./results", exist_ok=True)

path = './UAE_Cifar_10'
model = torch.load(path + '.model', weights_only=False, map_location='cpu')
model.eval()

To_Uniform(
  (encoder): SimpleEncoder(
    (conv1): Conv2d(3, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(256, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv4): Conv2d(256, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc): Sequential(
      (0): Linear(in_features=1024, out_features=512, bias=True)
      (1): ReLU()
      (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Linear(in_features=512, out_features=256, bias=True)
      (4): 

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [20]:
# "results" klasörünü oluştur (zaten varsa hata vermez)
os.makedirs("results", exist_ok=True)

flow_name = './results/UAE_Latent_FM'
flow = Flow().to(device)

optimizer = torch.optim.Adam(flow.parameters(), 0.0003)
loss_fn = nn.MSELoss()

In [21]:
# --- 1. Veriyi encode et (z, y) olarak döndür ---
def get_encoded_loader(data_loader, encoder, device='cpu'):
    encoder.eval()
    encoded = []
    labels  = []

    with torch.no_grad():
        for x, y in data_loader:  # CIFAR-10: (görüntü, etiket)
            x = x.to(device)
            z = encoder(x)
            encoded.append(z.cpu())
            labels.append(y.cpu())

    all_z = torch.cat(encoded, dim=0)   # Latent vektörler
    all_y = torch.cat(labels, dim=0)    # Sınıf etiketleri

    dataset = TensorDataset(all_z, all_y)
    return DataLoader(dataset, batch_size=data_loader.batch_size, shuffle=True)

# Modeli cihazda tut
model.encoder.to(device)

# Encode edilmiş latent + etiket veri yükleyicileri
encoded_train_loader = get_encoded_loader(train_loader, model.encoder, device=device)
encoded_val_loader   = get_encoded_loader(val_loader, model.encoder, device=device)

In [22]:
train_cost = []
val_cost = []
best_val_loss = float('inf')

for epoch in range(10000):
    epoch_losses = []

    flow.train()
    for z1, _ in encoded_train_loader:  # etiket kullanılmıyor ama alınmalı
        z1 = z1.to(device)
        z0 = torch.rand_like(z1)
        t = torch.rand(len(z1), 1, device=device)

        zt = (1 - t) * z0 + t * z1
        dzt = z1 - z0

        optimizer.zero_grad()
        pred = flow(t=t, x_t=zt)
        loss = loss_fn(pred, dzt)
        loss.backward()
        optimizer.step()

        epoch_losses.append(loss.item())

    mean_train_loss = sum(epoch_losses) / len(epoch_losses)
    train_cost.append(mean_train_loss)

    # Validation
    flow.eval()
    val_losses = []
    with torch.no_grad():
        for z1_val, _ in encoded_val_loader:  # yine etiket dahil ama kullanılmıyor
            z1_val = z1_val.to(device)
            z0_val = torch.rand_like(z1_val)
            t_val = torch.rand(len(z1_val), 1, device=device)

            zt_val = (1 - t_val) * z0_val + t_val * z1_val
            dzt_val = z1_val - z0_val

            pred_val = flow(t=t_val, x_t=zt_val)
            val_loss = loss_fn(pred_val, dzt_val)
            val_losses.append(val_loss.item())

    mean_val_loss = sum(val_losses) / len(val_losses)
    val_cost.append(mean_val_loss)

    # En iyi modeli kaydet
    if mean_val_loss < best_val_loss:
        print("Saved!")
        best_val_loss = mean_val_loss
        torch.save(flow, flow_name + '.model')

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/1000 - Train Loss: {mean_train_loss:.4f} - Val Loss: {mean_val_loss:.4f}")

# Son modeli kaydet
torch.save(flow, flow_name + '_final.model')


Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Epoch 10/1000 - Train Loss: 0.1125 - Val Loss: 0.1124
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Epoch 20/1000 - Train Loss: 0.1097 - Val Loss: 0.1093
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Epoch 30/1000 - Train Loss: 0.1075 - Val Loss: 0.1073
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Epoch 40/1000 - Train Loss: 0.1057 - Val Loss: 0.1056
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Epoch 50/1000 - Train Loss: 0.1038 - Val Loss: 0.1039
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Epoch 60/1000 - Train Loss: 0.1023 - Val Loss: 0.1021
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Epoch 70/1000 - Train Loss: 0.1010 - Val Loss: 0.1010
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Epoch 80/1000 - Train Loss: 0.0997 - Val Loss: 0.0996
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Epoch 90/1000 - Train Loss: 0.0985 - Val Loss: 0.0986
Sav

In [23]:
# CSV dosyasına kaydet
train_losses = train_cost  # liste veya numpy array
val_losses = val_cost

np.savetxt("./results/losses.csv",
           np.column_stack((train_losses, val_losses)),
           delimiter=",",
           header="train_loss,val_loss",
           comments="")